In [1]:
from os import listdir as ls, path, makedirs
import math
import json
import argparse
from natsort import natsorted
import cv2
from itertools import product
import pandas as pd
import numpy as np
from moviepy.editor import VideoFileClip
import re
import csv

In [2]:
FPS = 29.97

In [3]:
split_home = r'/beegfs/.global0/ws/sapo684c-sac_space/split_test_trial.json'
with open(split_home, 'r') as file:
    split = json.load(file)
train = split['train']
val = split['val']
test = split['test']
csv_home=r'/projects/p_scads_bdai/ellen_show/labels/'
na_layer = r'/projects/p072/p_scads_bdai/ellen_show/NA-layer'

In [4]:
all_csvs = natsorted([elem for elem in ls(csv_home) if elem.endswith('csv')])

In [5]:
node_path = r'/beegfs/.global0/ws/sapo684c-sac_space/NA_layer/Ellen_dataset_trial/'
base_path = r'/projects/p_scads_bdai/ellen_show/'
OF_path = r'/projects/p072/p_scads_bdai/ellen_show/publish/'
videos_home= r'/projects/p_scads_bdai/ellen_show/full_dataset_30_videos_yao_and_suwei/30_videos/'
keypoints_home = path.join(base_path, 'publish')
length_msec = 1300
thr = 0.55
user_input='yes'
user_input =user_input.lower()
pos_keywords= ['yes','true']
OF_option_enabled= any(keyword in user_input for keyword in pos_keywords)

In [6]:
home_path = node_path + "/" + \
            "ellen_show_length_" + str(length_msec) + \
            "_iou_thr_" + str(thr) + "/"

In [7]:
for elem in product([home_path], ['train', 'val', 'test'], ['gesture', 'nongesture']):
    makedirs(path.join(*elem), exist_ok=True)

In [8]:
def sec2frame(sec):
    return math.ceil(sec * FPS)

In [9]:
def framelist2tensor(frame_list, video_shape, output_shape):
    tensor = np.zeros((len(frame_list), *output_shape[::-1]))
    rescale_x = output_shape[1]/video_shape[1]
    rescale_y = output_shape[0]/video_shape[0]
    limit_x = output_shape[1]
    limit_y = output_shape[0]
    for i, elem in enumerate(frame_list):
        data = json.load(open(elem))
        people = data['people']
        for person in people:
            keypoints = person['pose_keypoints_2d']
            y = np.array(keypoints[::3]) * rescale_y
            x = np.array(keypoints[1::3]) * rescale_x
            y = np.clip(y, 0, limit_y - 1)
            x = np.clip(x, 0, limit_x - 1)
            for x_coord, y_coord in zip(x,y):
                if x_coord == 0 and y_coord == 0:
                    continue
                tensor[i, round(x_coord), round(y_coord)] = 1
    return tensor

In [10]:
def get_label(csv_data, start_interval, end_interval, thr):
    length = 0
    for i in range(len(csv_data)):
        start_time = csv_data['Begin Time - msec'].iloc[i]
        end_time = csv_data['End Time - msec'].iloc[i]
        if start_interval <= start_time < end_interval <= end_time:
            length += (end_interval - start_time)
        if start_interval <= start_time < end_time <= end_interval:
            length += (end_time - start_time)
        if start_time <= start_interval <= end_time:
            length += (min(end_interval, end_time) - start_interval)
        if start_time > end_interval:
            break
    iou = length/(end_interval - start_interval)
    if iou > thr:
        return True
    else:
        return False

In [11]:
def subclip2tensor(subclip,OF_frame_list,start_frame,end_frame,num_frames):
    frame_tensor = np.zeros((num_frames, 320, 320, 3))
    OF_frames = np.zeros((num_frames, 320, 320, 3))
    OF_frame_tensor = np.zeros((num_frames, 320, 320, 6))
    curr_frame = start_frame
    #print("start and end",start_frame,end_frame)
    
    #print(OF_frame_list)
    frame_iterator = subclip.iter_frames()
    for i, frame in enumerate(frame_iterator):
        frame_tensor[i] = cv2.resize(frame, (320, 320))
        #print('frame_tensor',i)
        pattern =r'(\d+)\.png'
        #print(pattern)
        if re.search(pattern, OF_frame_list[i]):
            #print(OF_frame_list[i])
            extracted_number = re.findall(pattern, OF_frame_list[i])[0]
            #print(extracted_number)
            if curr_frame == int(extracted_number) -1:
                #print("current_frame",curr_frame)
                OF_frame = cv2.imread(OF_frame_list[i])
                OF_frames[i] = cv2.resize(OF_frame, (320,320))
                OF_frame_tensor[i] = np.concatenate ((frame_tensor[i], OF_frames[i]), axis=2)
        if curr_frame!=end_frame:
            curr_frame +=1
            #print('curr_frame',curr_frame)

    #print(OF_frame_tensor[0])
    return frame_tensor,OF_frame_tensor

In [12]:
def get_frame_list(frame_list, start_frame, end_frame, num_frames):
    overhead = 0
    if math.ceil(end_frame - start_frame) != num_frames:
        overhead = (start_frame + num_frames - end_frame)
        end_frame += overhead
    return frame_list[start_frame:end_frame]

In [13]:
def create_dataset(split, regime):
    dataset_ind = 0
    clip_length = length_msec/1000
    num_frames = int(math.ceil(clip_length*29.97))
    for elem in split:
        case_name = elem.split('.')[0]
        csv_file = [elem for elem in all_csvs if elem.startswith(case_name)][0]
        csv_data = pd.read_csv(path.join(csv_home,csv_file))
        csv_na = path.join(na_layer,csv_file)
        with open(csv_na, mode='r') as csv_file:
            csv_reader = csv.reader(csv_file)
            next(csv_reader)
            first_row = next(csv_reader, None)
            if first_row is None:
                print(f"CSV file {csv_na} is empty no data rows)")
                continue
        
        csv_data = csv_data[csv_data['Duration - msec']>=150]
        csv_data = csv_data.reset_index()
        video_path = path.join(videos_home, elem)
        video_obj = VideoFileClip(video_path)
        video_duration = np.floor(video_obj.duration).astype('int')
        #print(video_duration)
        i = 0
        start_msec = 0
        all_files_home = path.join(keypoints_home, case_name, case_name+'_json')
        all_files = natsorted(ls(all_files_home))
        OF_file = path.join(OF_path,case_name, case_name+ '_OF',case_name+'_OF_SSR_1')
        OF_file_all =natsorted(ls(OF_file))
        while start_msec/1000 + 2*clip_length < video_duration:
            start_sec, end_sec = i * clip_length, (i + 1) * clip_length
            start_msec = start_sec * 1000
            end_msec = end_sec * 1000
            print(start_msec,end_msec)
            
            
            with open(csv_na, mode='r') as csv_file:
                csv_reader = csv.reader(csv_file)
                next(csv_reader)
                print('yes')
                print(csv_reader)
                process_flag = True
                for row in csv_reader:
                    print('row',row)
                    start_time = int(row[1])
                    end_time = int(row[2])
                    print('STart and end time',start_time, end_time)
                    if not (start_msec < start_time and end_msec < start_time) and not (start_msec > end_time and end_msec > end_time):
                        process_flag = False
                        print(f"This belongs to the na layer ({start_time}, {end_time}")
                        break
                if process_flag:
                    start_frame = sec2frame(i*clip_length)
                    end_frame = sec2frame((i+1)*clip_length)
                    #print(start_frame,end_frame)
                    frame_list = get_frame_list(all_files, start_frame, end_frame, num_frames)
                    frame_list = [path.join(all_files_home, elem) for elem in frame_list]
                    OF_frame_list = get_frame_list(OF_file_all, start_frame, end_frame, num_frames)
                    OF_frame_list = [path.join(OF_file, elem)for elem in OF_frame_list]
                    keypoints_tensor = framelist2tensor(frame_list, video_obj.size, (320,320))
                    label = get_label(csv_data, start_msec, end_msec, thr)   
                    subclip = video_obj.subclip(i*clip_length, (i+1)*clip_length)
                    clip_tensor, OF_clip_tensor = subclip2tensor(subclip,OF_frame_list,start_frame,end_frame, num_frames)

                    if OF_option_enabled:
                        input_item = {
                            'video': clip_tensor.astype('uint8'),
                            'keypoints': keypoints_tensor.astype('uint8'),
                            'video_OF':OF_clip_tensor.astype('uint8'),
                        }
                    else:
                        input_item = {
                            'video': clip_tensor.astype('uint8'),
                            'keypoints': keypoints_tensor.astype('uint8'),
                        }
                    elem_splitted = elem.split('_')
                    suffix = elem_splitted[0] + '_' + elem_splitted[-1].split('.')[0]
                    suffix1 =suffix + f'_{int(start_msec)}_{int(end_msec)}'
                    filename = '0'*(9 - len(str(dataset_ind))) + str(dataset_ind) + '_' + suffix1
                    if label:
                        filename = home_path + regime + '/gesture/' + filename
                    else:
                        filename = home_path + regime + '/nongesture/' + filename
                    np.savez_compressed(filename, **input_item)
                    dataset_ind += 1
                    i += 1
                else:
                    i += 1
                    continue
                    

In [19]:
def create_dataset(split, regime):
    dataset_ind = 0
    clip_length = length_msec/1000
    num_frames = int(math.ceil(clip_length*29.97))
    for elem in split:
        case_name = elem.split('.')[0]
        csv_file = [elem for elem in all_csvs if elem.startswith(case_name)][0]
        csv_data = pd.read_csv(path.join(csv_home,csv_file))
        csv_na = path.join(na_layer,csv_file)
        
        csv_data = csv_data[csv_data['Duration - msec']>=150]
        csv_data = csv_data.reset_index()
        video_path = path.join(videos_home, elem)
        video_obj = VideoFileClip(video_path)
        video_duration = np.floor(video_obj.duration).astype('int')
        #print(video_duration)
        i = 0
        start_msec = 0
        all_files_home = path.join(keypoints_home, case_name, case_name+'_json')
        all_files = natsorted(ls(all_files_home))
        OF_file = path.join(OF_path,case_name, case_name+ '_OF',case_name+'_OF_SSR_1')
        OF_file_all =natsorted(ls(OF_file))
        while start_msec/1000 + 2*clip_length < video_duration:
            start_sec, end_sec = i * clip_length, (i + 1) * clip_length
            start_msec = start_sec * 1000
            end_msec = end_sec * 1000
            print(start_msec,end_msec) 
            with open(csv_na, mode='r') as csv_file:
                csv_reader = csv.reader(csv_file)
                next(csv_reader)
                first_row = next(csv_reader, None)
                if first_row is None:
                    print(f"CSV file {csv_na} is empty no data rows)")
                    
                    
                    start_frame = sec2frame(i*clip_length)
                    end_frame = sec2frame((i+1)*clip_length)
                    #print(start_frame,end_frame)
                    frame_list = get_frame_list(all_files, start_frame, end_frame, num_frames)
                    frame_list = [path.join(all_files_home, elem) for elem in frame_list]
                    OF_frame_list = get_frame_list(OF_file_all, start_frame, end_frame, num_frames)
                    OF_frame_list = [path.join(OF_file, elem)for elem in OF_frame_list]
                    keypoints_tensor = framelist2tensor(frame_list, video_obj.size, (320,320))
                    label = get_label(csv_data, start_msec, end_msec, thr)   
                    subclip = video_obj.subclip(i*clip_length, (i+1)*clip_length)
                    clip_tensor, OF_clip_tensor = subclip2tensor(subclip,OF_frame_list,start_frame,end_frame, num_frames)

                    if OF_option_enabled:
                        input_item = {
                            'video': clip_tensor.astype('uint8'),
                            'keypoints': keypoints_tensor.astype('uint8'),
                            'video_OF':OF_clip_tensor.astype('uint8'),
                        }
                    else:
                        input_item = {
                            'video': clip_tensor.astype('uint8'),
                            'keypoints': keypoints_tensor.astype('uint8'),
                        }
                    elem_splitted = elem.split('_')
                    suffix = elem_splitted[0] + '_' + elem_splitted[-1].split('.')[0]
                    suffix1 =suffix + f'_{int(start_msec)}_{int(end_msec)}'
                    filename = '0'*(9 - len(str(dataset_ind))) + str(dataset_ind) + '_' + suffix1
                    if label:
                        filename = home_path + regime + '/gesture/' + filename
                    else:
                        filename = home_path + regime + '/nongesture/' + filename
                    np.savez_compressed(filename, **input_item)
                    dataset_ind += 1
                    i += 1
                    
                else:
                    
                    
                    print('yes')
                    csv_file.seek(0)
                    csv_reader1 = csv.reader(csv_file)
                    next(csv_reader1)
                    process_flag = True
                    for row in csv_reader1:
                        print('row',row)
                        start_time = int(row[1])
                        end_time = int(row[2])
                        print('STart and end time',start_time, end_time)
                        if not (start_msec < start_time and end_msec < start_time) and not (start_msec > end_time and end_msec > end_time):
                            process_flag = False
                            print(f"This belongs to the na layer ({start_time}, {end_time}")
                            break
                    if process_flag:
                        start_frame = sec2frame(i*clip_length)
                        end_frame = sec2frame((i+1)*clip_length)
                        #print(start_frame,end_frame)
                        frame_list = get_frame_list(all_files, start_frame, end_frame, num_frames)
                        frame_list = [path.join(all_files_home, elem) for elem in frame_list]
                        OF_frame_list = get_frame_list(OF_file_all, start_frame, end_frame, num_frames)
                        OF_frame_list = [path.join(OF_file, elem)for elem in OF_frame_list]
                        keypoints_tensor = framelist2tensor(frame_list, video_obj.size, (320,320))
                        label = get_label(csv_data, start_msec, end_msec, thr)   
                        subclip = video_obj.subclip(i*clip_length, (i+1)*clip_length)
                        clip_tensor, OF_clip_tensor = subclip2tensor(subclip,OF_frame_list,start_frame,end_frame, num_frames)

                        if OF_option_enabled:
                            input_item = {
                                'video': clip_tensor.astype('uint8'),
                                'keypoints': keypoints_tensor.astype('uint8'),
                                'video_OF':OF_clip_tensor.astype('uint8'),
                            }
                        else:
                            input_item = {
                                'video': clip_tensor.astype('uint8'),
                                'keypoints': keypoints_tensor.astype('uint8'),
                            }
                        elem_splitted = elem.split('_')
                        suffix = elem_splitted[0] + '_' + elem_splitted[-1].split('.')[0]
                        suffix1 =suffix + f'_{int(start_msec)}_{int(end_msec)}'
                        filename = '0'*(9 - len(str(dataset_ind))) + str(dataset_ind) + '_' + suffix1
                        if label:
                            filename = home_path + regime + '/gesture/' + filename
                        else:
                            filename = home_path + regime + '/nongesture/' + filename
                        np.savez_compressed(filename, **input_item)
                        dataset_ind += 1
                        i += 1
                    else:
                        i += 1
                        continue
                    

In [ ]:
if __name__ == "__main__":
    print("Start creating train dataset!")
    create_dataset(split["train"], "train")
    print("Start creating val dataset!")
    create_dataset(split["val"], "val")
    print("Start creating test dataset!")
    create_dataset(split["test"], "test")